In [ ]:
# ! pip install ~/repos/_ml

In [ ]:
models = {
    'stsnet-v013-swish': {
        'input_tensors': [
           "age_scaled",
           "carshock",
           "chf",
           "chrlungd",
           "classnyh",
           "creatlst_scaled",
           "cva",
           "cvd",
           "cvdpcarsurg",
           "cvdtia",
           "diabetes",
           "dialysis",
           "ethnicity",
           "gender",
           "hct_scaled",
           "hdef_scaled",
           "heightcm_scaled",
           "hypertn",
           "immsupp",
           "incidenc",
           "infendty",
           "medadp5days",
           "medgp",
           "medinotr",
           "medster",
           "numdisv",
           "platelets_scaled",
           "pocpci",
           "pocpciin",
           "prcab",
           "prcvint",
           "prvalve",
           "pvd",
           "raceasian",
           "raceblack",
           "racecaucasian",
           "racenativeam",
           "raceothernativepacific",
           "resusc",
           "status",
           "vdinsufa",
           "vdinsufm",
           "vdinsuft",
           "vdstena",
           "vdstenm",
           "wbc_scaled",
           "weightkg_scaled"
        ],
        'output_tensors': [
            'sts_death',
        ],
    },
}

In [ ]:
import os
import sys

from ml4c3.recipes import infer_multimodal_multitask
from ml4c3.arguments import parse_args

num_bootstraps = 10
data_splits = ['test']
models_folder = os.path.expanduser('~/dropbox/sts-ecg/results')
subcohorts = ['', 'cab', 'cab-valve', 'valve', 'other']
output_folder = os.path.expanduser('~/dropbox/sts-ecg/predictions')

In [ ]:
for subcohort in subcohorts:
    subcohort = f'{"-" if subcohort != "" else ""}{subcohort}'
    bootstraps_folder = os.path.expanduser(f'~/dropbox/sts-data/bootstraps{subcohort}')
    for bootstrap in range(num_bootstraps):
        bootstrap = str(bootstrap)
        for split in data_splits:
            split_csv = f"{split}.csv"
            split_csv = os.path.join(bootstraps_folder, bootstrap, split_csv)
            for model, io in models.items():
                input_tensors = io["input_tensors"]
                output_tensors = io["output_tensors"]
                sys.argv = f"""
                .
                --mode infer
                --model_file {os.path.join(models_folder, model, bootstrap, "model_weights.h5")}
                --tensors /storage/shared/ecg/mgh
                --patient_csv {os.path.expanduser('~/dropbox/sts-data/sts-mgh.csv')}
                --test_csv {split_csv}
                --input_tensors {" ".join(input_tensors)}
                --output_tensors {" ".join(output_tensors)}
                --optimizer adam
                --batch_size 64
                --output_folder {os.path.join(output_folder, f"{model}{subcohort}")}
                --id {bootstrap}
                """.split()
                args = parse_args()
                infer_multimodal_multitask(args)